# 第4回：クレンジング・特徴量作成
* 今回は、機械学習の活用に向けてのデータクレンジング・特徴量作成を行います。
* クレンジング
    * リーク情報であることが判明したdurationを削除します
    * month（月）が英字表記のため数値に変換します
    * データの前から順番に2008,2009,2010年のデータとなっていますが、年を表すyear列がないため追加します。
* scikit-learn用の数値変換
    * 機械学習のアルゴリズムを適用するためには、文字列形式のカテゴリデータやNaN形式を数値に変換する必要があります。

# 1. データ読み込み

* 前回同様に、実行したいセルを選択して「Shift」+「Enter」でプログラムを実行できます。

In [ ]:
# ライブラリ（様々な機能を簡単に使えるもの）のインポート
# このセルは、実行してもセルの下になにも表示されません
import pandas as pd             # 表形式のデータ操作ライブラリ
import matplotlib.pyplot as plt # グラフ描画ライブラリ
import seaborn as sns           # グラフ描画ライブラリ

In [ ]:
data = pd.read_csv("bank-full.csv")
data.head()

# 2. クレンジング
## 2.1. リーク情報の削除
* 前回リーク情報であることが分かったduration（通話時間）を削除しておきます

In [ ]:
data = data.drop('duration', axis=1) # dropで'duration'列を削除

In [ ]:
data.head() # データを表示し、duration列が削除されていることを確認

## 2-2. 月を数値に変換
* month列は"jan", "feb"のように文字データとなっていますが、これも数値化しましょう。
* "jan"が1、"feb"が2、... となるように変換します。

In [ ]:
# 月データを文字から数値に変換
data['month'] = data['month'].map( {'jan': 1, 'feb': 2, 'mar': 3, 'apr': 4, 'may': 5, 'jun': 6, 
                                'jul': 7, 'aug': 8, 'sep': 9, 'oct': 10, 'nov': 11, 'dec': 12} ).astype(int)

In [ ]:
data.head() # 変換後のデータを表示し、month列が数値に変換されたことを確認

In [ ]:
# 各月のデータ件数を表示
data['month'].value_counts()

* もともとmonthの先頭5行は"may"でしたが、正しく5に変換できました。

## 2.3. 年を表す列を追加
* 今回のデータの説明文を読むと、"bank-full.csv with all examples, ordered by date (from May 2008 to November 2010)" と記載があるので、2008年5月から2010年11月までのデータが順番に並んでいそうです。
* 少しトリッキーですが、横軸にデータの行番号、縦軸にmonthをとる折れ線グラフで可視化して実際に様子を確認します。

In [ ]:
sns.set() # グラフのデザインをSeabornライブラリのデフォルトスタイルに変更
sns.set_context('talk') # グラフの字の大きさを調整
plt.figure(figsize=(20, 8))
sns.lineplot(x=range(data.shape[0]), y='month', data=data) #lineplotで折れ線グラフを描画

* グラフが規則的に階段状になっていることから、データが時系列的に（年月に沿って順番に）並んでいることがわかります。
    * 2008年のデータが一番多く、2009年、2010年とだんだんデータが減っていることがわかります。
* 具体的に何行目までが2008年なのか、何行目から何行目までが2009年、何行目以降が2010年なのかを確認しましょう。
* 2008年と2009年の境目はおよそ20000行目～30000行目の間に存在し、2009年と2010年の境目は40000行目～50000行目の間に存在しそうです。そこで、それぞれの範囲でデータを抜き出してから月日を小さい順番に並べて、月日がもっとも小さい日（12月と1月の境目）の行数を確認しましょう。

In [ ]:
# 2008年と2009年の境目確認
data.loc[20000:30000,:].sort_values(['month', 'day']).head()

* 20000行目から3000行目で月日の数値が一番小さいデータは27729行目であることから、27728行目は12月データ（つまり2008年12月のデータ）で、27729行目以降が2009年1月データであることがわかります。

In [ ]:
# 2008年と2009年の境目確認
data.loc[40000:50000,:].sort_values(['month', 'day']).sort_index().head()

* データの27728行目までが2008年、27729行目から42590行目までが2009年、42591行目以降が2010年であることがわかります。
* 確認した行数をもとに実際にデータに "year" 列を追加してみましょう。

In [ ]:
# 'year' 列を追加
data.loc[0:27728, 'year']=2008
data.loc[27729:42590, 'year']=2009
data.loc[42591:, 'year']=2010

data['year'] = data['year'].astype(int) # yearは整数型に変換

#### year列が追加できたことを確認
* データを表示し、year列が追加されたこと、12月から1月になる境目でyear列の数値が変わることを確認します。

In [ ]:
data.head()

In [ ]:
# 2008年と2009年の境目
data.loc[27727:27730, :]

In [ ]:
# 2009年と2010年の境目
data.loc[42589:42592, :]

* 無事にyear列を追加できました。

# 3. 変換したデータをCSVファイルで保存
* 以上の処理によりすべてのデータを数値化できたので、無事に機械学習を実行できる形になりました。
* 何度も同じ処理をしないで済むように、変換後のデータをCSVファイルで保存してみましょう。
* to_csv() 機能を使うと簡単にCSVファイルで保存できます。
* 今回は、"data_clensing.csv" という名前で保存してみます。

In [ ]:
data.to_csv("data_clensing.csv", index=False) # to_csv() でデータをCSVファイルとして出力

* みなさんのフォルダに、"data_clensing.csv"というファイルができたことを確認してみて下さい。